In [2]:
import sagemaker

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/dataset"

# role = "arn:aws:iam::235411143540:role/SageMakerRole"# claude_perrin_3
# role = "arn:aws:iam::146258905784:role/SageMakerRole"# claude_perrin_5
role = "arn:aws:iam::224769526787:role/SageMakerRole"# claude_perrin_9

print(bucket)

sagemaker.config INFO - Not applying SDK defaults from location: /opt/homebrew/share/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/viktor/Library/Application Support/sagemaker/config.yaml
sagemaker-eu-central-1-224769526787


In [3]:
# inputs = sagemaker_session.upload_data(path="saved_dataset", bucket=bucket, key_prefix=prefix)
# print("input spec (in this case, just an S3 path): {}".format(inputs))
# Train set and Test set
# common_bucket_name = "s3://sagemaker-eu-central-1-235411143540"
# train = (f"{common_bucket_name}/train_directory")
# test = (f"{common_bucket_name}/test_dir" )

common_bucket_name = f"s3://{bucket}"

# train = (f"{common_bucket_name}/crop_train_set")
# test = (f"{common_bucket_name}/crop_test_set")
train = (f"{common_bucket_name}/train_set")
train = (f"{common_bucket_name}/cropped_train")
test = (f"{common_bucket_name}/cropped_train")
print(train,'\n',test)

s3://sagemaker-eu-central-1-224769526787/cropped_train 
 s3://sagemaker-eu-central-1-224769526787/cropped_train


In [3]:
!pygmentize code/faster_rcnn.py


import argparse
import os
import boto3


#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.ops import nms
from dataset_helper import save_test_img, save_to_bucket, create_output_bucket
from dataset_helper import get_train_data_loader, get_test_data_loader
import json
from torch.optim.lr_scheduler import StepLR

from torch.nn.functional import pad
from conf import *

def get_object_detection_model(num_classes=NUMBER_OF_CLASSES):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model


class Main:
    def __init__(self, args):
        self.args =  args
        self.s3_session 

In [4]:
from sagemaker.pytorch import PyTorch
instance_count = 2
if instance_count > 1:
    distribution={"torch_distributed": {"enabled": True}} #torch_distributed
else:
    distribution = None

estimator = PyTorch(
    entry_point="train.py",
    source_dir='.',        # Directory where your source code is located
    role=role,
    py_version="py310",
    framework_version="2.0.1",
    instance_count=instance_count,
    # distribution=distribution,
    instance_type="ml.m4.xlarge", # ml.g4dn.xlarge ml.m4.xlarge	
    hyperparameters={"epochs": 2, "backend": "gloo"},
)


sagemaker.config INFO - Not applying SDK defaults from location: /opt/homebrew/share/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/viktor/Library/Application Support/sagemaker/config.yaml


In [5]:
estimator.fit({"train": train, "test": test})


Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-12-08-13-49-17-721


2023-12-08 13:49:20 Starting - Starting the training job......
2023-12-08 13:50:05 Starting - Preparing the instances for training......
2023-12-08 13:51:20 Downloading - Downloading input data......
2023-12-08 13:51:55 Training - Downloading the training image......
2023-12-08 13:53:21 Uploading - Uploading generated training modelbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-12-08 13:53:16,335 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-12-08 13:53:16,336 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-08 13:53:16,336 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-12-08 13:53:16,348 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-12-08 13:53:16,350 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-

UnexpectedStatusException: Error for Training job pytorch-training-2023-12-08-13-49-17-721: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "ModuleNotFoundError: No module named 'training'"
Command "/opt/conda/bin/python3.10 train.py --backend gloo --epochs 2", exit code: 1

In [32]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")


INFO:sagemaker:Repacking model artifact (s3://sagemaker-eu-central-1-235411143540/pytorch-training-2023-10-27-19-14-02-631/output/model.tar.gz), script artifact (s3://sagemaker-eu-central-1-235411143540/pytorch-training-2023-10-27-19-14-02-631/source/sourcedir.tar.gz), and dependencies ([]) into single tar.gz file located at s3://sagemaker-eu-central-1-235411143540/pytorch-training-2023-10-27-19-49-51-655/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-training-2023-10-27-19-49-51-655
INFO:sagemaker:Creating endpoint-config with name pytorch-training-2023-10-27-19-49-51-655
INFO:sagemaker:Creating endpoint with name pytorch-training-2023-10-27-19-49-51-655


-------!

In [34]:
import gzip
import numpy as np
import random
import os

data_dir = "data/MNIST/raw"
with gzip.open(os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"), "rb") as f:
    images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28).astype(np.float32)

mask = random.sample(range(len(images)), 16)  # randomly select some of the test images
mask = np.array(mask, dtype=np.int32)
data = images[mask]


In [35]:
response = predictor.predict(np.expand_dims(data, axis=1))
print("Raw prediction result:")
print(response)
print()

labeled_predictions = list(zip(range(10), response[0]))
print("Labeled predictions: ")
print(labeled_predictions)
print()

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print("Most likely answer: {}".format(labeled_predictions[0]))


Raw prediction result:
[[    0.         -1531.03552246  -775.55126953  -882.36639404
  -1270.45324707  -764.815979    -803.70916748 -1069.17114258
   -846.71520996 -1144.79272461]
 [ -355.86022949  -440.81268311  -202.45556641  -266.35440063
   -401.78820801  -287.02746582  -264.79299927  -576.97924805
      0.          -450.95666504]
 [ -800.54821777  -702.08032227  -719.37127686  -437.40423584
    -19.32922363  -343.06335449  -536.19616699  -357.53234863
   -276.92120361     0.        ]
 [ -645.06445312  -817.94506836  -521.99169922  -694.60449219
      0.          -513.83740234  -381.3157959   -398.30310059
   -555.9921875   -121.46380615]
 [ -357.71896362  -962.51599121 -1015.74230957  -680.15093994
   -666.33557129     0.          -534.38995361  -856.23693848
   -422.42626953  -573.88012695]
 [ -819.02593994 -1104.11669922  -689.6842041  -1209.65405273
   -659.09802246  -546.53356934     0.         -1378.28796387
   -805.45489502 -1012.93432617]
 [ -484.87252808  -614.40661621  -4

In [37]:
sagemaker_session.delete_endpoint(endpoint_name=predictor.endpoint_name)


INFO:sagemaker:Deleting endpoint with name: pytorch-training-2023-10-27-19-49-51-655


ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "pytorch-training-2023-10-27-19-49-51-655".